Cornology case- LP Modeling

Team 15_MBAN1

February 2022

In [3]:
#!/usr/bin/env python

get_ipython().system('pip install pulp # install pulp')

##########################################
######## Linear-Programming MODEL ########
##########################################

#########Importing necessary packaging for Linear-Programming MODEL ##########
import pulp as p
import pandas as pd

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/katesu/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/wk/y9rdrwj92177jxmt5lc4_7vm0000gn/T/2c2dc655143b4ef181c6fc3343c9cfc9-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/wk/y9rdrwj92177jxmt5lc4_7vm0000gn/T/2c2dc655143b4ef181c6fc3343c9cfc9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 39 RHS
At line 41 BOUNDS
At line 50 ENDATA
Problem MODEL has 1 rows, 8 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.30232e+09 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -1.30232e+09 to -1.79769e+308
Probing was tried 0 times and created 0 

In [4]:
######### <LpProblem()> to create the maximization problem "LP cor model"#####
lp_mo = p.LpProblem("LP_cor_model", p.LpMaximize)

################## <p.LpVariable> to create the decision variables ###########

NE_YRD   = p.LpVariable("NE_YRD", cat='Binary')    # Making binary-decision variable from entry port data of Nong Khai 
NE_NC_WC = p.LpVariable("NE_NC_WC", cat='Binary')  # Making binary-decision variable from entry port data of Nong Khai  
YRD_NE   = p.LpVariable("YRD_NE", cat='Binary')    # Making binary-decision variable from entry prot data Bangkok 
NC_SE    = p.LpVariable("NC_SE", cat='Binary')     # Making binary-decision variable from entry port data Chiang Rai
NC_NE_WC = p.LpVariable("NC_NE_WC", cat='Binary')  # Making binary-decision variable from entry port data Chiang Rai 
WC_SC    = p.LpVariable("WC_SC", cat='Binary')     # Making binary-decision variable from entry port data Yala 
WC_NC_NE = p.LpVariable("WC_NC_NE", cat='Binary')  # Making binary-decision variable from entry port data Yala 
NC_SC    = p.LpVariable("NC_SC", cat='Binary')     # Making binary-decision variable from entry port data Chiang Rai 


In [5]:
######################### Creating factory-cost Variables #####################

Non_Kha_fac_cost    = 11252161    # The cost of production for Nong Khai
Chi_Rai_fac_cost    = 10998001+(0.56*500000)+(0.7*600000)-(0.1*150000)        
                                       # The cost of production for Chiang Rai 
Ban_fac_cost        = 24062500    # The cost of production for Bangkok 
Yala_fac_cost       = 10303601    # The cost of production for Yala


In [6]:
######################### Creating demand Variable ############################

NEast_D_Ori     = 0           # The demand of original-North east (units)
NEast_D_bla     = 70          # The demand of blast- North east (units)

YRD_D_ori       = 180         # The demand of original- YRD (units)
YRD_D_bla       = 112         # The demand of blast- YRD (units)

SEast_D_ori     = 127         # The demand of original- South east (units)
SEast_D_bla     = 37          # The demand of blast- South east (units)

NCen_D_ori      = 90          # The demand of original- North central (units)
NCen_D_bla      = 70          # The demand of blast- North central (units)

SCen_D_ori      = 118         # The demand of original- South central (units)
SCen_D_bla      = 70          # The demand of blast- South central (units)

WChina_D_ori    = 90          # The demand of original- Western China (units)
WChina_D_bla    = 0           # The demand of blast- Western China (units)

Ori_profit      = 12500       # Profit per unit of original sold
Bla_profit      = 7500        # Profit per unit of blast sold


In [7]:
#####################Profit for the all the paths ########################

Profit_NEast  = NEast_D_Ori * Ori_profit + NEast_D_bla * Bla_profit    # Northeast path profit
Profit_YRD    = YRD_D_ori * Ori_profit + YRD_D_bla * Bla_profit        # YRD path profit
Profit_SEast  = SEast_D_ori * Ori_profit + SEast_D_bla * Bla_profit    # SouthEast path profit
Profit_NCen   = NCen_D_ori * Ori_profit + NCen_D_bla * Bla_profit      # NorthCentral path profit
Profit_SCen   = SCen_D_ori * Ori_profit + SCen_D_bla * Bla_profit      # SouthCentral path profit
Profit_WChina = WChina_D_ori * Ori_profit + WChina_D_bla * Bla_profit  # WesternChina path profit

In [8]:
############################# Objective function ###########################

lp_mo += NE_YRD * ((Profit_NEast + Profit_YRD) * 360 - Non_Kha_fac_cost) + NE_NC_WC * ((Profit_NEast + Profit_NCen + Profit_WChina) * 360 - Non_Kha_fac_cost) + YRD_NE * ((Profit_YRD + Profit_NEast) * 360 - Ban_fac_cost) + NC_SE * ((Profit_NCen + Profit_SEast) * 360 - Chi_Rai_fac_cost) + NC_SC * ((Profit_NCen + Profit_SCen) * 360 - Chi_Rai_fac_cost) + NC_NE_WC * ((Profit_NCen + Profit_NEast + Profit_WChina) * 360 - Chi_Rai_fac_cost) + WC_SC * ((Profit_WChina + Profit_SCen) * 360 - Yala_fac_cost) + WC_NC_NE * ((Profit_WChina + Profit_NCen + Profit_NEast) * 360 - Yala_fac_cost)



In [9]:
################################ constraints #################################
########################## 5 runs of limited for all the demand###############


lp_mo += NE_YRD + NE_NC_WC + YRD_NE + NC_SE + NC_SC + NC_NE_WC + WC_SC + WC_NC_NE == 1 # Only one possible path


In [10]:
############################# Soving the problem #############################

status = lp_mo.solve()    # solving the model
print(p.LpStatus[status])   # print the solved model
print(lp_mo) # Display the problem 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/katesu/opt/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/wk/y9rdrwj92177jxmt5lc4_7vm0000gn/T/c5ea056116e04df7a238a2d823bbda3e-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/wk/y9rdrwj92177jxmt5lc4_7vm0000gn/T/c5ea056116e04df7a238a2d823bbda3e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 39 RHS
At line 41 BOUNDS
At line 50 ENDATA
Problem MODEL has 1 rows, 8 columns and 8 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.30232e+09 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -1.30232e+09 to -1.79769e+308
Probing was tried 0 times and created 0 

In [11]:
######################### Printing objective value ###########################
print("Objective function value is: ", p.value(lp_mo.objective))

Objective function value is:  1302316999.0


In [12]:
############################ Printing the variables #########################
for variable in lp_mo.variables():
    print(" {} = {} ".format(variable.name, variable.varValue))

 NC_NE_WC = 0.0 
 NC_SC = 1.0 
 NC_SE = 0.0 
 NE_NC_WC = 0.0 
 NE_YRD = 0.0 
 WC_NC_NE = 0.0 
 WC_SC = 0.0 
 YRD_NE = 0.0 
